In [70]:
# Importaciones y configuración global
# Todas las librerías se importan aquí de forma única
from pathlib import Path
import warnings
import numpy as np
import pandas as pd

# Opciones de visualización de pandas
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 120)

# Directorio de salida relativo a este notebook
OUTPUT_DIR = Path('./nhanes_clean')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Directorio de salida: {OUTPUT_DIR.resolve()}")


Directorio de salida: /workspace/noteebook/nhanes_clean


In [71]:
# Carga de datos
# Forzamos dtype=str inicialmente para evitar inferencias erróneas; luego tipificamos
# Se usa ruta relativa directa al CSV desde la carpeta `noteebook/`
raw_df = pd.read_csv('../NHANES2009-2012.csv', dtype=str, na_values=['', 'NA', 'NaN', 'null', 'None'])
print(raw_df.shape)
raw_df.head(3)


(10000, 75)


,SurveyYr,ID,Gender,Age,AgeDecade,AgeMonths,Race1,Race3,Education,MaritalStatus,HHIncome,HHIncomeMid,Poverty,HomeRooms,HomeOwn,Work,Weight,Length,HeadCirc,Height,BMI,BMICatUnder20yrs,BMI_WHO,Pulse,BPSysAve,BPDiaAve,BPSys1,BPDia1,BPSys2,BPDia2,BPSys3,BPDia3,Testosterone,DirectChol,TotChol,UrineVol1,UrineFlow1,UrineVol2,UrineFlow2,Diabetes,DiabetesAge,HealthGen,DaysPhysHlthBad,DaysMentHlthBad,LittleInterest,Depressed,nPregnancies,nBabies,Age1stBaby,SleepHrsNight,SleepTrouble,PhysActive,PhysActiveDays,TVHrsDay,CompHrsDay,TVHrsDayChild,CompHrsDayChild,Alcohol12PlusYr,AlcoholDay,AlcoholYear,SmokeNow,Smoke100,Smoke100n,SmokeAge,Marijuana,AgeFirstMarij,RegularMarij,AgeRegMarij,HardDrugs,SexEver,SexAge,SexNumPartnLife,SexNumPartYear,SameSex,SexOrientation
0,2009_10,55829,female,28,20-29,343,White,NaN,CollegeGrad,Married,more 99999,100000,5,5,Own,Working,61,NaN,NaN,161.8,23.3,NaN,18.5_to_24.9,82,121,79,124,78,124,74,118,84,NaN,2.79,4.14,215,3.909,NaN,NaN,No,NaN,Vgood,0,3,NaN,NaN,NaN,NaN,NaN,7,No,Yes,2,NaN,NaN,NaN,NaN,Yes,3,72,NaN,No,Non-Smoker,NaN,Yes,15,No,NaN,Yes,Yes,13,20,1,No,Heterosexual
1,2009_10,57112,male,14,10-19,170,White,NaN,NaN,NaN,75000-99999,87500,4.17,4,Own,NaN,88.9,NaN,NaN,162.3,33.75,NaN,30.0_plus,70,102,62,102,62,NaN,NaN,NaN,NaN,NaN,1.09,2.79,98,NaN,NaN,NaN,No,NaN,Excellent,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009_10,60232,male,80,NaN,NaN,White,NaN,8thGrade,Married,20000-24999,22500,1.58,6,Own,NotWorking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84,141,57,142,62,138,58,144,56,NaN,1.4,4.22,121,0.59,NaN,NaN,No,NaN,Poor,30,0,NaN,NaN,NaN,NaN,NaN,9,No,No,NaN,NaN,NaN,NaN,NaN,Yes,NaN,0,NaN,No,Non-Smoker,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# Estandarización de nombres de columnas

def standardize_column_name(name: str) -> str:
    name = name.strip()
    name = name.replace('\n', ' ')
    # Reemplazos comunes
    replacements = {
        '%': 'pct',
        '#': 'num',
        '/': ' ',
        '\\': ' ',
        '(': ' ',
        ')': ' ',
        ',': ' ',
        ';': ' ',
        ':': ' ',
        '.': ' ',
        '-': ' ',
    }
    for k, v in replacements.items():
        name = name.replace(k, v)
    # Normalización
    name = ' '.join(name.split())  # colapsar espacios
    name = name.lower()
    name = name.replace(' ', '_')
    return name

raw_df.columns = [standardize_column_name(c) for c in raw_df.columns]
print(len(raw_df.columns), 'columnas estandarizadas')
raw_df.columns[:10]


75 columnas estandarizadas


Index(['surveyyr', 'id', 'gender', 'age', 'agedecade', 'agemonths', 'race1', 'race3', 'education', 'maritalstatus'], dtype='object')

In [73]:
# Tipificación de columnas (heurística)
from pandas.api.types import is_numeric_dtype


def try_parse_numeric(series: pd.Series) -> pd.Series:
    # Intento de conversión a numérico, preservando NaN
    converted = pd.to_numeric(series.str.replace(',', '.', regex=False), errors='coerce')
    return converted


def infer_and_cast_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    df_casted = df.copy()
    for col in df_casted.columns:
        s = df_casted[col]
        # Casos booleanos y categorías típicas
        lower_vals = s.dropna().str.lower().unique().tolist()[:50]
        lower_vals_set = set(lower_vals)
        bool_tokens = {'yes', 'no', 'true', 'false', 'y', 'n', 'si', 'sí', '0', '1'}
        if lower_vals_set <= bool_tokens:
            mapping = {
                'yes': True, 'y': True, 'true': True, 'si': True, 'sí': True, '1': True,
                'no': False, 'n': False, 'false': False, '0': False
            }
            df_casted[col] = s.str.lower().map(mapping)
            continue
        
        # Fechas: probar formatos comunes antes de fallback genérico
        common_formats = ['%Y-%m-%d', '%d/%m/%Y', '%m/%d/%Y']
        parsed = None
        for fmt in common_formats:
            parsed_try = pd.to_datetime(s, format=fmt, errors='coerce')
            if parsed_try.notna().mean() >= 0.8:
                parsed = parsed_try
                break
        if parsed is None:
            with warnings.catch_warnings():
                warnings.simplefilter('ignore', category=UserWarning)
                parsed = pd.to_datetime(s, errors='coerce')
        if parsed.notna().mean() >= 0.8 and parsed.dropna().between('1900-01-01', '2100-12-31').all():
            df_casted[col] = parsed
            continue
        
        # Numéricos
        numeric_series = try_parse_numeric(s)
        # Heurística: si al menos 80% es convertible y hay > 5 valores no nulos
        if numeric_series.notna().mean() >= 0.8 and numeric_series.notna().sum() >= 5:
            df_casted[col] = numeric_series
            continue
        
        # Caso general: dejar como string (object)
    return df_casted


df = infer_and_cast_dtypes(raw_df)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 75 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   surveyyr          10000 non-null  object 
 1   id                10000 non-null  int64  
 2   gender            10000 non-null  object 
 3   age               10000 non-null  int64  
 4   agedecade         9667 non-null   object 
 5   agemonths         4962 non-null   object 
 6   race1             10000 non-null  object 
 7   race3             5000 non-null   object 
 8   education         7221 non-null   object 
 9   maritalstatus     7231 non-null   object 
 10  hhincome          9189 non-null   object 
 11  hhincomemid       9189 non-null   float64
 12  poverty           9274 non-null   float64
 13  homerooms         9931 non-null   float64
 14  homeown           9937 non-null   object 
 15  work              7771 non-null   object 
 16  weight            9922 non-null   float64

In [74]:
# Configuración de reglas de limpieza (ajustables)
CATEGORICAL_MAX_UNIQUE_RATIO = 0.05   # si #categorías únicas / filas < 5% -> candidata a categoría
NUMERIC_OUTLIER_METHOD = 'iqr'        # 'iqr' o None
IQR_CAP_FACTOR = 1.5                  # 1.5 para IQR clásico
DROP_COLS_NA_RATIO_THRESHOLD = 0.50   # eliminar columnas con >50% de NaN

NAs_COMMON_TOKENS = {'n/a', 'na', 'nan', 'null', 'none', 'missing', 'desconocido'}


In [75]:
# Limpieza de valores nulos y normalización de tokens vacíos

before_na = df.isna().sum().sum()
# Normalizar strings tipo 'NA', 'N/A', etc. a NaN para columnas object
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].replace({v: np.nan for v in NAs_COMMON_TOKENS}, regex=False)

# 1) Eliminación de columnas con alto porcentaje de nulos
na_ratio = df.isna().mean()  # proporción de NaN por columna
cols_to_drop = na_ratio[na_ratio > DROP_COLS_NA_RATIO_THRESHOLD].index.tolist()
prev_cols = df.shape[1]
df = df.drop(columns=cols_to_drop)
print({'dropped_null_ratio_cols': cols_to_drop, 'prev_cols': prev_cols, 'new_cols': df.shape[1]})

after_normalization_na = df.isna().sum().sum()
print({'na_before': int(before_na), 'na_after_token_norm': int(after_normalization_na)})

# Imputación: solo numéricos con mediana; categóricas se dejan en NaN
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

# Guardamos métricas previas
na_counts_before = df.isna().sum().to_dict()

for c in num_cols:
    if df[c].isna().any():
        median_val = df[c].median()
        df[c] = df[c].fillna(median_val)

na_counts_after = df.isna().sum().to_dict()
print('NA por columna (antes -> después) en algunas columnas:')
for k in list(df.columns)[:10]:
    print(k, '->', na_counts_before.get(k, 0), 'to', na_counts_after.get(k, 0))


{'dropped_null_ratio_cols': ['agemonths', 'length', 'headcirc', 'bmicatunder20yrs', 'testosterone', 'urinevol2', 'urineflow2', 'diabetesage', 'littleinterest', 'depressed', 'npregnancies', 'nbabies', 'age1stbaby', 'physactivedays', 'tvhrsday', 'comphrsday', 'tvhrsdaychild', 'comphrsdaychild', 'alcoholday', 'smokenow', 'smokeage', 'marijuana', 'agefirstmarij', 'regularmarij', 'ageregmarij', 'sexnumpartyear', 'sexorientation'], 'prev_cols': 75, 'new_cols': 48}
{'na_before': 279722, 'na_after_token_norm': 84998}
NA por columna (antes -> después) en algunas columnas:
surveyyr -> 0 to 0
id -> 0 to 0
gender -> 0 to 0
age -> 0 to 0
agedecade -> 333 to 333
race1 -> 0 to 0
race3 -> 5000 to 5000
education -> 2779 to 2779
maritalstatus -> 2769 to 2769
hhincome -> 811 to 811


In [76]:
# Eliminación de duplicados exactos
initial_rows = len(df)
df = df.drop_duplicates()
removed = initial_rows - len(df)
print({'initial_rows': int(initial_rows), 'duplicates_removed': int(removed), 'final_rows': int(len(df))})


{'initial_rows': 10000, 'duplicates_removed': 3221, 'final_rows': 6779}


In [77]:
# Normalización de categorías comunes (minúsculas, stripping, mapeos simples)

def normalize_category_value(val):
    if pd.isna(val):
        return val
    if isinstance(val, str):
        v = val.strip().lower()
        mapping = {
            'male': 'male', 'm': 'male', 'masculino': 'male',
            'female': 'female', 'f': 'female', 'femenino': 'female',
        }
        return mapping.get(v, v)
    return val

for c in df.columns:
    if df[c].dtype == 'object':
        df[c] = df[c].map(normalize_category_value)

print('Normalización de categorías aplicada a columnas tipo object')


Normalización de categorías aplicada a columnas tipo object


In [78]:
# Detección y tratamiento de outliers (cap por IQR)
# Convertimos de forma segura a numérico y aplicamos cap solo en columnas mayormente numéricas
if NUMERIC_OUTLIER_METHOD == 'iqr':
    for c in df.columns:
        # Omitir columnas booleanas
        if pd.api.types.is_bool_dtype(df[c]):
            continue
        # Forzar conversión a numérico; valores no numéricos -> NaN
        s_num = pd.to_numeric(df[c], errors='coerce')
        valid_mask = s_num.notna()
        # Saltar columnas con poca cobertura numérica o muy pocos datos válidos
        if valid_mask.mean() < 0.8 or valid_mask.sum() < 5:
            continue
        # Cuantiles con numpy para evitar problemas de dtype
        arr = s_num[valid_mask].to_numpy(dtype='float64', copy=False)
        q1 = np.nanpercentile(arr, 25)
        q3 = np.nanpercentile(arr, 75)
        iqr = q3 - q1
        if not np.isfinite(iqr) or iqr == 0:
            continue
        lower = q1 - IQR_CAP_FACTOR * iqr
        upper = q3 + IQR_CAP_FACTOR * iqr
        df[c] = s_num.clip(lower, upper)
print('Outliers tratados con IQR' if NUMERIC_OUTLIER_METHOD == 'iqr' else 'Tratamiento de outliers desactivado')


Outliers tratados con IQR


In [79]:
# Métricas finales y guardado

metrics = {
    'rows': int(len(df)),
    'cols': int(len(df.columns)),
    'num_cols': int(sum(pd.api.types.is_numeric_dtype(df[c]) for c in df.columns)),
    'cat_cols': int(sum(not pd.api.types.is_numeric_dtype(df[c]) for c in df.columns)),
}
print(metrics)

# Guardado
csv_path = OUTPUT_DIR / 'NHANES2009-2012_clean.csv'
parquet_path = OUTPUT_DIR / 'NHANES2009-2012_clean.parquet'

df.to_csv(csv_path, index=False)
try:
    df.to_parquet(parquet_path, index=False)
    print({'saved_csv': str(csv_path), 'saved_parquet': str(parquet_path)})
except Exception as e:
    print({'saved_csv': str(csv_path), 'parquet_error': str(e)})


{'rows': 6779, 'cols': 48, 'num_cols': 22, 'cat_cols': 26}
{'saved_csv': 'nhanes_clean/NHANES2009-2012_clean.csv', 'saved_parquet': 'nhanes_clean/NHANES2009-2012_clean.parquet'}
